## Explanation of the Code
### Class Initialization:

- The __init__ method initializes the parameters:
- depth_radius: The number of neighboring channels on each side of the current channel to include in the normalization window. A radius of 5, for example, covers 2 channels on each side, plus the central channel, resulting in a window  size of 5.
- bias: A constant added to the normalization term to prevent division by zero.
- alpha: A scaling factor applied to the local sum of squared values.
- beta: The exponent applied to the normalization term.
### Forward Pass (Normalization):

- Input Shape: This implementation expects input in the shape (batch_size, channels, height, width), which is standard for convolutional outputs.
- Normalization Process: For each batch item and each channel:
- It calculates the start and end indices of the channels in the window, based on the depth_radius parameter.
- It computes the sum of squares of inputs within this window to form the normalization term.
- Finally, the normalized output is calculated by dividing each input by this normalization term, raised to the power of beta.
- Output Shape: The output is in the same shape as the input.
### Example Usage:

- Instantiate the custom_LocalResponseNormalization class and apply it to input X using forward(X).
- This allows LocalResponseNormalization to be applied to a sample input batch, yielding an output that highlights the effect of normalization across channels.
### Key Differences from TensorFlow's tf.nn.local_response_normalization and PyTorch's nn.LocalResponseNorm
- Custom Parameters: This code allows setting custom depth_radius, bias, alpha, and beta, similar to TensorFlow and PyTorch, but it’s implemented from scratch.
- Parameter Values: You can adjust the depth_radius, alpha, beta, and bias based on specific requirements or layer settings in your model.
- Manual Channel Normalization: Unlike built-in implementations that handle parallelized operations, this custom code iterates manually over batches and channels, which may affect efficiency.


## INBUILT LRN

In [12]:
import tensorflow as tf

# Sample input tensor
X = tf.random.normal([1,96,55,55])

# Apply Local Response Normalization
depth_radius = 5
bias = 1.0
alpha = 1e-4
beta = 0.75

lrn_output = tf.nn.local_response_normalization(X, depth_radius=depth_radius, bias=bias, alpha=alpha, beta=beta)
lrn_output[0,0,1,1]

<tf.Tensor: shape=(), dtype=float32, numpy=0.04559753090143204>

## CUSTOM LRN

In [13]:
import numpy as np

class custom_LocalResponseNormalization:
    def __init__(self, depth_radius=5, bias=1.0, alpha=1e-4, beta=0.75):
        """
        Initializes the LocalResponseNormalization layer.

        Parameters:
        - depth_radius (int): Number of neighboring channels for normalization.
        - bias (float): Added to the normalization term to avoid division by zero.
        - alpha (float): Scaling parameter.
        - beta (float): Exponent for normalization.
        """
        self.depth_radius = depth_radius
        self.bias = bias
        self.alpha = alpha
        self.beta = beta

    def forward(self, X):
        """
        Applies local response normalization to the input data.

        Parameters:
        - X (np.array): Input data with shape (batch_size, channels, height, width).

        Returns:
        - np.array: The normalized data.
        """
        N, C, H, W = X.shape  # batch size, channels, height, width
        squared_input = np.square(X)
        output = np.zeros_like(X)

        for n in range(N):  # Iterate over batch size
            for c in range(C):  # Iterate over channels
                # Define the start and end of the depth radius range
                start = max(0, c - self.depth_radius // 2)
                end = min(C, c + self.depth_radius // 2 + 1)

                # Sum the square of inputs within the local region
                local_sum = np.sum(squared_input[n, start:end, :, :], axis=0)
                
                # Apply LRN formula
                scale = self.bias + (self.alpha * local_sum)
                output[n, c, :, :] = X[n, c, :, :] / (scale ** self.beta)

        return output

lrn=custom_LocalResponseNormalization()
lrn_output=lrn.forward(X)
lrn_output[0,0,1,1]


0.04560225